# Field Analysis


In [ ]:
%config InlineBackend.figure_format = 'retina' # Nicer plots

In [ ]:
from math import pi

import h5py
import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy.constants import c, e, epsilon_0

from genesis.version4 import Genesis4, Write

h = scipy.constants.value("Planck constant in eV/Hz")
Z0 = pi * 119.916983  # V^2 / m

# Create field data

In [ ]:
%%time
G = Genesis4("data/basic4/cu_hxr.in", verbose=True)
G.input.main.time.sample = 200
G.input.main.track.zstop = 40
G.input.main.namelists.append(Write(field="end"))
G.nproc = 0  # auto-select
output = G.run()

In [ ]:
G.path

In [ ]:
G.plot("field_energy", y2=["field_xsize", "field_ysize"], ylim2=[0, 100e-6])

In [ ]:
G.verbose = True
output.load_fields()

# 3D Field data

The full field data is stored as a 3D array of complex numbers `DFL` in units of `sqrt(W)`.

The relation of this and the electric field `E` in V/m is:
```
E = DFL * sqrt(2*Z0) / Δ
```
Where `Z0 = π * 119.9169832 V^2/W` exactly and `Δ` is the grid spacing. 

In [ ]:
DFL = output.field3d["end"].dfl  # sqrt(W)

DFL.dtype, DFL.shape

In [ ]:
param = output.field3d["end"].param

param

Gather some convenient variables and arrays:

In [ ]:
Ns = param.slicecount
ds = param.slicespacing
λ0 = param.wavelength
f0 = c / λ0
dt = ds / c
Δ = param.gridsize

s = np.arange(0, Ns) * ds
t = -s / c

# Field power

The power array sums over the x and y components of the absolute square of the field data.

In [ ]:
power0 = G.output["field_power"][-1, :]  # W

In [ ]:
power1 = np.sum(np.sum(np.abs(DFL) ** 2, axis=0), axis=0)  # W

These are the same:

In [ ]:
np.allclose(power0, power1)

In [ ]:
fig, ax = plt.subplots()
ax.plot(s * 1e6, power0 / 1e9)
ax.set_xlabel("s (µm)")
ax.set_ylabel("power (GW)")

# Field energy

The total field energy is the integral:

In [ ]:
energy1 = np.trapz(power1, dx=dt)

energy1  # J

In [ ]:
np.sum(power1) * dt  # J

# On-axis field intensity and phase

In [ ]:
field = G.output.field
intensity0 = field.intensity_nearfield[-1, :]  # W/m^2
phase0 = field.phase_nearfield[-1, :]  # radian

In [ ]:
icenter = param.gridpoints // 2
field1 = DFL[icenter, icenter, :]  # sqrt(W)
phase1 = np.angle(field1)  # radian

intensity1 = np.abs(field1**2) / Δ**2  # W/m^2

These are the same:

In [ ]:
np.allclose(intensity0, intensity1)

In [ ]:
np.allclose(phase0, phase1)

In [ ]:
fig, ax = plt.subplots()
ax.plot(s * 1e6, intensity0)
ax.set_xlabel("s (µm)")
ax.set_ylabel(r"intensity (W/m$^2$)")

The same field can be reconstructed from these arrays

In [ ]:
field0 = np.sqrt(intensity0) * np.exp(1j * phase0) * Δ  # sqrt(W)

These are the same:

In [ ]:
np.allclose(field0, field1)

# Spectral fluence

The spectrum calculation takes some care with the FFT and units.

In [ ]:
def spectrum_from_field(field, dt=1):
    """
    Calculates the spectrum (fourier transformed field)
    from a complex field array with spacing dt.

    Parameters
    ----------
    field: nd.array of shape (n,)
        Complex field

    dt: float
        Spacing of the field data in some units (e.g. 's')

    Returns
    -------
    freqs: nd.array of shape (n,)
        Frequencies in reciprocal space with inverse units (e.g. 'Hz = 1/s')

    spectrum: nd.array of shape (n,)
        The fourier transformed field

    """
    if len(field.shape) != 1:
        raise ValueError("Only 1D arrays are currently supported")
    spectrum = np.fft.fftshift(np.fft.fft(field)) * dt

    ns = len(field)
    freqs = np.fft.fftshift(np.fft.fftfreq(ns, dt))

    return freqs, spectrum

In [ ]:
freqs, spectrum = spectrum_from_field(field0, dt)

# Frequency spacing
# df = np.diff(freqs)[0] # Hz

df = 1 / (dt * len(field0))

df

Check Plancherel theorem

In [ ]:
np.sum(np.abs(field0) ** 2) * dt  # J

In [ ]:
np.sum(np.abs(spectrum) ** 2) * df  # J

In [ ]:
freqs, spectrum = spectrum_from_field(field0, dt)

fig, ax = plt.subplots()

photon_energies = h * (freqs + f0)  # eV
spectral_fluence = np.abs(spectrum**2) / Δ**2 / h  # J/m^2/eV

ax.plot(photon_energies, spectral_fluence)
ax.set_xlabel("photon energy (eV)")
ax.set_ylabel(r"spectral fluence (J/m$^2$/eV)")

In [ ]:
fig, ax = plt.subplots()
J_per_photon = h * (freqs + f0) * e  # note h is in eV/Hz
ax.plot(photon_energies, spectral_fluence / J_per_photon * 1e-6)
ax.set_xlabel("photon energy (eV)")
ax.set_ylabel(r"spectral intensity (photons/mm$^2$/eV)")

# 3D FFT

WORK IN PROGRESS

In [ ]:
field3d = DFL * np.sqrt(2 * Z0) / Δ  # Electric field
# field3d = np.pad(field3d, 100, constant_values = 0) #Pad
nx, ny, nz = field3d.shape
dx, dy, dz = Δ, Δ, ds
nx, ny, nz, dx, dy, dz

# reciprocal spacings
dkx, dky, dkz = 1 / (nx * dx), 1 / (ny * dy), 1 / (nz * dz)

In [ ]:
spectrum3d = np.fft.fftshift(np.fft.fftn(field3d)) * dx * dy * dz

In [ ]:
# Check total energy integral
np.sum(np.abs(field3d) ** 2) * dx * dy * dz * epsilon_0 / 2

In [ ]:
np.sum(np.abs(spectrum3d) ** 2) * dkx * dky * dkz * epsilon_0 / 2

In [ ]:
kx = np.fft.fftshift(np.fft.fftfreq(nx, dx))
ky = np.fft.fftshift(np.fft.fftfreq(ny, dy))
kz = np.fft.fftshift(np.fft.fftfreq(nz, dz))

In [ ]:
Kx, Ky, Kz = np.meshgrid(kx, ky, kz, indexing="ij")
kz0 = 2 * pi / λ0
Kz = Kz + kz0

K = np.sqrt(Kx**2 + Ky**2 + Kz**2)
k = K.flatten()
dEk = np.abs(spectrum3d.flatten()) ** 2 * dkx * dky * dkz * epsilon_0 / 2

In [ ]:
np.sum(dEk)

In [ ]:
x = k * c * h / (2 * pi)  # eV
w = dEk
bins = 1000

hist, bin_edges = np.histogram(x, bins=bins, weights=w)
hist_x = bin_edges[:-1] + np.diff(bin_edges) / 2
hist_width = np.diff(bin_edges)

hist_y = hist / hist_width
# hist_y, hist_f, hist_prefix = nice_array(hist/hist_width)

fig, ax = plt.subplots()
ax.bar(hist_x, hist_y * 1e6, hist_width, color="gray")

ax2 = ax.twinx()
ax2.plot(hist_x, np.cumsum(hist) * 1e6)
ax2.set_ylabel("cumulative energy (µJ)")
ax2.set_ylim(0, None)

ax.set_xlabel("photon energy (eV)")
ax.set_ylabel("spectral energy (µJ/eV)")

# Field data formats

Genesis4 writes a custom field format. LUME-Genesis provides a reader for this, as well as a conversion tool to write in the [openPMD-wavefront standard](https://github.com/openPMD/openPMD-standard/blob/upcoming-2.0.0/EXT_Wavefront.md).

## Read Field h5

In [ ]:
G.output.load_fields()

In [ ]:
end_field = G.output.field3d["end"]
DFL = end_field.dfl
PARAM = end_field.param

In [ ]:
DFL.shape, DFL.dtype, PARAM

## Write Wavefront in openPMD-wavefront

In [ ]:
end_field.write_openpmd_wavefront("genesis4_wavefront.h5", verbose=True)

In [ ]:
# Read back

with h5py.File("genesis4_wavefront.h5", "r") as h5:
    print(list(h5["data"]["000000"]["meshes"]))

    g = h5["data/000000/meshes/electricField"]
    print(dict(g.attrs))

    # Get
    E2 = h5["data/000000/meshes/electricField/x"][:]

In [ ]:
# Data is the same
np.allclose(E2, DFL)

# Plot

Simple plot

In [ ]:
# Sum over y and compute the absolute square
dat2 = np.sum(np.abs(DFL) ** 2, axis=1)

In [ ]:
# Plot the center
plt.figure(figsize=(12, 8))
plt.imshow(dat2[::, :], aspect="auto")
# plt.axis('off');plt.savefig('../../assets/field.png', bbox_inches='tight')

In [ ]:
def plot_slice(i=0):
    dat = np.angle(DFL[:, :, i])
    dx = PARAM.gridsize * 1e6

    plt.figure(figsize=(8, 8))
    plt.xlabel("x (µm)")
    plt.xlabel("y (µm)")
    plt.title(f"Phase for slice {i}")
    plt.imshow(dat.T, origin="lower", extent=[-dx, dx, -dx, dx])


plot_slice(i=100)

# Interactive 

In [ ]:
try:
    from ipywidgets import interact
except ImportError:
    pass
else:
    interact(plot_slice, i=(0, len(DFL[0, 0, :]) - 1, 1))

# Cleanup 

In [ ]:
import os

os.remove("genesis4_wavefront.h5")